#### Requirements

In [1]:
import sys
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LinearRegression

#### Constants

In [3]:
# General
BASE_PATH = os.path.dirname(os.getcwd())

# Mapping
CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}
CHANGE_STATUS_MAP = {'Greenland': 0, 'Land Cleared': 1, 'Excavation': 1, 'Materials Dumped': 3, 'Prior Construction': 3, 'Materials Introduced': 4, 'Construction Started': 5, 'Construction Midway': 6, 'Construction Done': 8, 'Operational': 10, None: None}

# Data
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']
GEOGRAPHY_TYPES = ['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
                   'Coastal', 'Lakes', 'Barren Land', 'Desert', 'Hills', 'Snow'] 
URBAN_TYPES = ['Sparse Urban', 'Rural', 'Dense Urban', 'Urban Slum', 'Industrial']

# Columns groups
COLUMNS_TO_DROP = ['geography_type', 'urban_type']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']
CHANGE_STATUS_COLUMNS = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4']
CHANGE_STATUS_VALUE_COLUMNS = ['change_status_value_date0', 'change_status_value_date1', 'change_status_value_date2', 'change_status_value_date3', 'change_status_value_date4']

# Output file
OUTPUT_FILE = 'preprocessed_train.geojson'

#### Data preprocessing

In [4]:
## Read data
train_df = gpd.read_file(f'{BASE_PATH}/data/train.geojson', index_col=0)
#test_df = gpd.read_file(f'{BASE_PATH}/data/test.geojson', index_col=0)

In [5]:
## One-hot encoding
for geograph_type in GEOGRAPHY_TYPES:
    train_df[geograph_type] = train_df['geography_type'].apply(lambda x: 1 if geograph_type in x else 0)
for urban_type in URBAN_TYPES:
    train_df[urban_type] = train_df['urban_type'].apply(lambda x: 1 if urban_type in x else 0)

In [ ]:
## Create new polygon features
train_df['area'] = train_df['geometry'].area
train_df['length'] = train_df['geometry'].length
train_df['centroid'] = train_df['geometry'].centroid

In [8]:
## Fix date
train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

## Create deltas: color and time
for metric in METRICS:
    for color in COLORS:
        for i in range(2, 6):
            delta = train_df[f'img_{color}_{metric}_date{i}'] - train_df[f'img_{color}_{metric}_date{i-1}']
            train_df[f'img_{color}_{metric}_delta{i}'] = delta
        train_df[f'img_{color}_{metric}_delta_total'] = train_df[f'img_{color}_{metric}_date5'] - train_df[f'img_{color}_{metric}_date1']
for i in range(1, 5):
    train_df[f'date_delta{i}'] = train_df[f'date{i}'] - train_df[f'date{i-1}']
train_df['date_delta_total'] = train_df[f'date4'] - train_df[f'date1']

In [9]:
## Map change_type
train_df['change_type'] = train_df['change_type'].map(CHANGE_TYPE_MAP)

In [ ]:
## Create change_status_values
train_df[CHANGE_STATUS_VALUE_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)

## Create civilization_rate
num_samples = train_df.shape[0]
coef = np.zeros((num_samples))
time_ctt = 1e9*60*90*24
ones = np.ones((5,1))
for i in range(num_samples):
    y = np.array(train_df[CHANGE_STATUS_VALUE_COLUMNS].iloc[i].astype(float))
    y_nan_mask = np.isnan(y)
    if y_nan_mask.all():
        continue
    y = y[:,np.newaxis]
    
    x = np.array(train_df[DATE_COLUMNS].iloc[i].astype(int))[:,np.newaxis]/time_ctt
    x = np.hstack((ones,x))
    
    W = np.linalg.inv(x[~y_nan_mask,:].T@x[~y_nan_mask,:])@x[~y_nan_mask,:].T@y[~y_nan_mask,:]
    coef[i] = W[1]
    #print(y, train_df["change_type"].iloc[i])
train_df["civilizating_rate"] = coef


In [20]:
## Drop uncessary columns
train_df = train_df.drop(columns=COLUMNS_TO_DROP)